In [1]:
import pandas as pd
from datetime import timedelta, date
import re
import requests
from bs4 import BeautifulSoup

In [2]:
page_num = [1,51]
soup=[]
for pagenums in page_num:
    page = requests.get(f'https://www.imdb.com/search/title/?title_type=feature&genres=crime&start={page_num}&explore=genres&ref_=adv_nxt')
    print(page)
    soup.append(BeautifulSoup(page.content,'html.parser'))

<Response [200]>
<Response [200]>


In [3]:
##movie_title and year of release
titles=[]
years= []
descriptions =[]
durations =[]
ratings =[]
votes = []
genres= []
stars = []
directors = []
for item in soup:
    movie_title = item.find_all('h3',class_="lister-item-header") #get the tag containing the movie title and year of release
    movie_discrip = item.find_all('div',class_="lister-item-content") #get tags containing the description


    #Movies and Years
    for movies in movie_title:
        titles.append(movies.text.replace("\n","").split(".")[1].split("(")[0])
        years.append( pd.Series(movies.text.replace('\n','')).str.extract(r'\((\d+)').squeeze()  )
    
      
    #Description
    for discrip in movie_discrip:
        ptags=discrip.find_all('p',class_="text-muted")
        discrips=ptags[1].text.replace('\n','')
        descriptions.append(discrips)
    #Duration
        duration =ptags[0].text.replace('\n','')
        durations.append( pd.Series(duration).str.extract(r'(\d+) min').squeeze())

    #Ratings
    rate = item.select(".lister-item-content")   
    for rates in rate:
        ratings.append( pd.Series(rates.text.replace('\n','')).str.extract(r'(\d+\.\d+)Rate').squeeze()  )
    #Votes
        votes.append( pd.Series(rates.text.replace('\n','')).str.extract(r'Votes\:(\d+,\d+)') .squeeze() )
        

    #Genres
    movie_genre = item.find_all('span',class_="genre")
    for genre in movie_genre:
        genres.append(genre.text.replace("\n","").strip('    '))

    #Stars
    movie_star = item.find_all('p',class_='')
    for star in movie_star:
        stars.append(star.text.replace("\n",'').split('Stars:')[1])
    #Directors
        directors.append(star.text.replace("\n",'').split('|')[0].split(':')[1])

        





In [9]:
start_date = []
end_date = []
for item in soup:
    
    result = item.find_all(class_ = "lister-item-header")
    for tag in result:
        url_codes = tag.a['href']
        url = 'https://www.imdb.com{}locations?ref_=ttfc_ql_5'.format(url_codes)
        response = requests.get(url)
        page = BeautifulSoup(response.content, 'html.parser')
        filming_dates = page.select('.listo')
        
        for tag in filming_dates:
            start_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'[fF]ilming [dD]ates\s+(\d+\s\w+\s\d+)').squeeze() )
            end_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'\d+\s-\s(\d+\s\w+\s\d+)').squeeze() )

In [5]:
start_date = []
end_date = []
for item in soup:
    
    result = item.find_all(class_ = "lister-item-header")
    for tag in result:
        partial_url = tag.a['href']
        url = 'https://www.imdb.com{}locations?ref_=ttfc_ql_5'.format(partial_url)
        response = requests.get(url)
        page = BeautifulSoup(response.content, 'html.parser')
        filming_stuff = page.select('.listo')
        
        for tag in filming_stuff:
            start_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'[fF]ilming [dD]ates\s+(\d+\s\w+\s\d+)').squeeze() )
            end_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'\d+\s-\s(\d+\s\w+\s\d+)').squeeze() )

In [11]:
years = [int(toint) for toint in years]

final_data =pd.DataFrame({'Title':titles, 'Descriptions':descriptions,'Release':years,'Filming_start_date':start_date,'Filming_end_date':end_date ,'directors':directors,'rating':ratings,'durations':durations,'votes':votes,'genres':genres,'stars':stars})
final_data.to_csv('Finaldata.csv', index=False)

In [12]:
final_data

,Title,Descriptions,Release,Filming_start_date,Filming_end_date,directors,rating,durations,votes,genres,stars
0,The Batman,"When the Riddler, a sadistic serial killer, be...",2022,27 January 2020,14 March 2020,Matt Reeves,8.4,176,"251,525","Action, Crime, Drama","Robert Pattinson, Zoë Kravitz, Jeffrey Wright,..."
1,West Side Story,"An adaptation of the 1957 musical, West Side S...",2021,1 July 2019,27 September 2019,Steven Spielberg,7.4,156,"52,704","Crime, Drama, Musical","Ansel Elgort, Rachel Zegler, Ariana DeBose, Da..."
2,The Weekend Away,A weekend getaway to Croatia that goes awry wh...,2022,NaN,NaN,Kim Farrant,5.6,89,"13,644","Crime, Drama, Mystery","Leighton Meester, Christina Wolfe, Ziad Bakri,..."
3,Yön ritari,When the menace known as the Joker wreaks havo...,2008,18 April 2007,11 November 2007,Christopher Nolan,9.1,152,"2,522","Action, Crime, Drama","Christian Bale, Heath Ledger, Aaron Eckhart, M..."
4,Nightmare Alley,An ambitious carny with a talent for manipulat...,2021,16 September 2020,11 December 2020,Guillermo del Toro,7.1,150,"85,126","Crime, Drama, Thriller","Bradley Cooper, Cate Blanchett, Toni Collette,..."
...,...,...,...,...,...,...,...,...,...,...,...
95,Cruella,A live-action prequel feature film following a...,2021,24 August 2019,26 November 2019,Craig Gillespie,7.4,134,"214,613","Adventure, Comedy, Crime","Emma Stone, Emma Thompson, Joel Fry, Paul Walt..."
96,Hiomattomat timantit,With his debts mounting and angry collectors c...,2019,NaN,NaN,"Benny Safdie, Josh Safdie",7.4,135,"261,581","Crime, Drama, Thriller","Adam Sandler, Julia Fox, Idina Menzel, Mesfin ..."
97,Uhrilampaat,A young F.B.I. cadet must receive the help of ...,1991,15 November 1989,1 March 1990,Jonathan Demme,8.6,118,"1,374","Crime, Drama, Thriller","Jodie Foster, Anthony Hopkins, Lawrence A. Bon..."
98,Leon,12-year-old Mathilda is reluctantly taken in b...,1994,1 June 1993,7 October 1993,Luc Besson,8.6,110,"1,117","Action, Crime, Drama","Jean Reno, Gary Oldman, Natalie Portman, Danny..."
